In [44]:
import pandas as pd
import numpy as np

In [2]:
# download table of drug resistance mutations
# https://github.com/hivdb/hivfacts/blob/main/data/drms_hiv1.csv

In [5]:
%%bash
wget https://raw.githubusercontent.com/hivdb/hivfacts/main/data/drms_hiv1.csv

--2024-01-05 11:55:48--  https://raw.githubusercontent.com/hivdb/hivfacts/main/data/drms_hiv1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3567 (3.5K) [text/plain]
Saving to: ‘drms_hiv1.csv’

     0K ...                                                   100% 28.1M=0s

2024-01-05 11:55:49 (28.1 MB/s) - ‘drms_hiv1.csv’ saved [3567/3567]



In [31]:
df_drms_hiv1 = pd.read_csv("drms_hiv1.csv")
df_drms_hiv1['position'] = df_drms_hiv1['position'].astype(int)

CA_positions = list(df_drms_hiv1[df_drms_hiv1['gene']=='CA']['position'])
PR_positions = list(df_drms_hiv1[df_drms_hiv1['gene']=='PR']['position'])
RT_positions = list(df_drms_hiv1[df_drms_hiv1['gene']=='RT']['position'])
IN_positions = list(df_drms_hiv1[df_drms_hiv1['gene']=='IN']['position'])

In [49]:
df = pd.read_csv("../../results/CAP217/week_358/snvs.annotated.csv")
df['n_reads_var'] = df['Rvar'] + df['Fvar']
df['coverage'] = df['Rtot'] + df['Ftot']
df['frequency'] = df['n_reads_var'] / df['coverage']
df["gene"].unique()
df['aa_position'] = df['aa_position'].apply(lambda x: np.nan if x=="error" else int(x))

In [51]:
df = df[['Pos', 'Ref', 'Var', 'frequency',  
         'AltAA_f1', 'RefAA_f1', 'f1_IsSynonymous',
         'AltAA_f2', 'RefAA_f2', 'f2_IsSynonymous',
         'AltAA_f3', 'RefAA_f3', 'f3_IsSynonymous',
          'gene', 'aa_position']]

In [79]:
# exclude CA_positions for now

df_res_PR = df[((df['gene']=='PR') & (df["aa_position"].isin(PR_positions)))]
df_res_PR = pd.merge(df_res_PR, df_drms_hiv1[df_drms_hiv1['gene']=='PR'], left_on="aa_position", right_on="position")
# PR corresponding to frame3 in our dataframe annotations

df_res_RT = df[((df['gene']=='RT') & (df["aa_position"].isin(RT_positions)))]
df_res_RT = pd.merge(df_res_RT, df_drms_hiv1[df_drms_hiv1['gene']=='RT'], left_on="aa_position", right_on="position")
# RT corresponding to frame3 in our dataframe annotations

df_res_IN = df[((df['gene']=='IN') & (df["aa_position"].isin(IN_positions)))]
df_res_IN = pd.merge(df_res_IN, df_drms_hiv1[df_drms_hiv1['gene']=='IN'], left_on="aa_position", right_on="position")
# IN corresponding to frame3 in our dataframe annotations
# however postion 232 has the wrong reference somehow 


df_res = pd.concat([df_res_PR, df_res_RT, df_res_IN])
df_res = df_res[['Pos', 'Ref', 'Var', 'frequency',
       'AltAA_f3', 'RefAA_f3', 'f3_IsSynonymous', 'gene_x', 'aa_position',
       'drug_class', 'gene_y', 'position', 'aa']]

In [80]:
df_res[df_res["aa"]==df_res["AltAA_f3"]]

,Pos,Ref,Var,frequency,AltAA_f3,RefAA_f3,f3_IsSynonymous,gene_x,aa_position,drug_class,gene_y,position,aa
12,2842,C,G,0.999038,G,A,0,RT,98.0,NNRTI,RT,98,G
13,4925,T,C,0.987421,N,N,1,IN,232.0,INSTI,IN,232,N


In [ ]:
# https://hivdb.stanford.edu/hivdb/by-patterns/report/?mutations=RT%3AA98G%2CIN%3AD232N&name=RT%3AA98G%2BIN%3AD232N 